In [65]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA  
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

### 1.Import required library

In [2]:
### 2.read the train and test csv 读取训练集和测试集文件
train_df = pd.read_csv('i239e_project_train.csv')
train_df.head(10)

,Feature#5,Feature#1,Feature#9,Feature#10,Name,Feature#2,Feature#8,Feature#4,Feature#7,Feature#3,Feature#6,Survived
0,1,425,NaN,2,"Rosblom, Mr. Viktor Richard",3,20.2125,18.0,370129,0,1,0
1,0,484,NaN,2,"Turkula, Mrs. (Hedwig)",3,9.5875,63.0,4134,1,0,1
2,0,419,NaN,2,"Matthews, Mr. William John",2,13.0000,30.0,28228,0,0,0
3,0,534,NaN,0,"Peter, Mrs. Catherine (Catherine Rizk)",3,22.3583,NaN,2668,1,2,1
4,0,67,F33,2,"Nye, Mrs. (Elizabeth Ramell)",2,10.5000,29.0,C.A. 29395,1,0,1
5,0,469,NaN,1,"Scanlan, Mr. James",3,7.7250,NaN,36209,0,0,0
6,1,254,NaN,2,"Lobb, Mr. William Arthur",3,16.1000,30.0,A/5. 3336,0,0,0
7,0,468,NaN,2,"Smart, Mr. John Montgomery",1,26.5500,56.0,113792,0,0,0
8,0,345,NaN,2,"Fox, Mr. Stanley Hubert",2,13.0000,36.0,229236,0,0,0
9,0,852,NaN,2,"Svensson, Mr. Johan",3,7.7750,74.0,347060,0,0,0


In [77]:
test_df = pd.read_csv('i239e_project_test.csv')
test_df.head(5)

test_names = test_df['Name'].values  ### Extract the names from the test set 提取出测试集中的名字

In [4]:
### 3.Select numerical features and discard difficult-to-handle text features 选择数据特征舍弃难处理的(比如缺失值太多的)特征以及文本特征。
train_df.info()
## AS we can see from info,there are so many missing value for Feature9,so discard
## And we also don't need 'Name'  and 'text'(F7) as feature

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741 entries, 0 to 740
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Feature#5   741 non-null    int64  
 1   Feature#1   741 non-null    int64  
 2   Feature#9   168 non-null    object 
 3   Feature#10  741 non-null    int64  
 4   Name        741 non-null    object 
 5   Feature#2   741 non-null    int64  
 6   Feature#8   741 non-null    float64
 7   Feature#4   599 non-null    float64
 8   Feature#7   741 non-null    object 
 9   Feature#3   741 non-null    int64  
 10  Feature#6   741 non-null    int64  
 11  Survived    741 non-null    int64  
dtypes: float64(2), int64(7), object(3)
memory usage: 69.6+ KB


In [5]:
drop_col = ['Name','Feature#9','Feature#7','Survived']   #We also dont need the result
numeric_col = [c for c in train_df.columns if c not in drop_col] #Select the left features

print(f'The features for PCA are:{numeric_col}')
print(f'The number of features:{len(numeric_col)}')

The features for PCA are:['Feature#5', 'Feature#1', 'Feature#10', 'Feature#2', 'Feature#8', 'Feature#4', 'Feature#3', 'Feature#6']
The number of features:8


In [63]:
#4.seperate the features and label 分离特征和标签
X_train_raw = train_df[numeric_col]  ##X_train_raw's frame:Dataframe
y_train_full = train_df['Survived'].values ##.values turn the list into the array, model process more fast and avoid the aligning problem by index

X_test_raw = test_df[numeric_col]

X_train_raw.head(5)

,Feature#5,Feature#1,Feature#10,Feature#2,Feature#8,Feature#4,Feature#3,Feature#6
0,1,425,2,3,20.2125,18.0,0,1
1,0,484,2,3,9.5875,63.0,1,0
2,0,419,2,2,13.0000,30.0,0,0
3,0,534,0,3,22.3583,NaN,1,2
4,0,67,2,2,10.5000,29.0,1,0


In [29]:
### 5.Data_cleaning 数据清洗
imputer =SimpleImputer(strategy='median')
#调用simpleImputer作为填充器准备填充

X_train_impute = imputer.fit_transform(X_train_raw) 
#设置好填充策略进行学习（统计中位数-浮点格式）并且填充
#fit 遍历每一列学习并把中位数存入内存 
#transform 把中位数填补到相应列中
#fit_transform以及transfrom 都是Scikit-learn中常用的学习应用方法

X_test_impute = imputer.transform(X_test_raw)  
#这里不再做测试集的中位数学习，而是直接填补训练集的中位数，避免带有测试时带有测试集的特征导致测试分数虚高
#我们要模拟的是“完全不知道未来数据”的情况。所以，所有的规则（中位数、平均值、标准化参数）都必须从训练集中产生。

#Scikit-learn 的底层运算是基于 NumPy 的。 
#为了计算效率和兼容性，大多数 Scikit-learn(机器学习算法库) 的工具（如 SimpleImputer, StandardScaler, PCA）在处理完数据后，
#都会默认把结果剥离成最原始的 NumPy 数组。

print(X_train_impute[:5])  #输出填充后的训练集数组前五行
print(X_train_impute.shape) #数组的信息（几行几列）
print(X_train_impute.ndim) #数组的维度
# shape和 ndim 都是针对数组的

#“数组”是计算机存储数据的一种容器格式，而“矩阵”是数学中的一种特定结构。
#scikit_learn的大多数工具处理完数据后默认以Numpy数组的形式保存。

[[  1.     425.       2.       3.      20.2125  18.       0.       1.    ]
 [  0.     484.       2.       3.       9.5875  63.       1.       0.    ]
 [  0.     419.       2.       2.      13.      30.       0.       0.    ]
 [  0.     534.       0.       3.      22.3583  28.       1.       2.    ]
 [  0.      67.       2.       2.      10.5     29.       1.       0.    ]]
(741, 8)
2


In [31]:
### 6.Data standardization 数据标准化
### 这一步在PCA之前必须得做，避免特征中过大的数据导致方差大，让PCA误判信息量大
### 标准化公式： z=(x-平均值u)/标准差σ  x-u的目的是中心化，把平均值作为坐标原点 
### 标准差 代表数据离平均值的平均远近程度，代表数据波动的实际尺寸。数据波动大，除以标准差把波动缩小，让数据标准化，让步长统一。
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train_impute) #对训练集数据进行标准化
#此时，平均值u和标准差σ已经固定
#scaler.mean_(平均值)以及scaler.var（方差）已经固定

X_test_scale = scaler.transform(X_test_impute)
#测试集用训练集的平均值和方差、标准差来计算。同样目的是为了避免数据泄露，训练结果虚高。

In [62]:
### 7.PCA Process （PCA特征工程处理，降维）
pca = PCA(n_components=5)

X_train_pca = pca.fit_transform(X_train_scale)
X_test_pca = pca.transform(X_test_scale)

print('PCA Process has been done successfully!')

print(f'Shape of X_train_pca is{X_train_pca.shape}') #观察训练集进行PCA主成分分析之后的结构
print(f"The variance ratio explained by PCA: {pca.explained_variance_ratio_}")  ### PCA之后的5个特征所含方差（信息量）的分别占比
print(f"The total retained information volume: {sum(pca.explained_variance_ratio_):.2%}") ### PCA之后5个特征的总方差(总信息量)占比

PCA Process has been done successfully!
Shape of X_train_pca is(741, 5)
The variance ratio explained by PCA: [0.23743356 0.21508452 0.12758895 0.11656596 0.10363509]
The total retained information volume: 80.03%


In [69]:
# 8.为了评估模型效果，我们将训练集切分为 训练集Traning Set(80%) 和 验证集Validation set(20%)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_pca, y_train_full, test_size=0.2, random_state=42 
    # test_size 为测试集比例 random_state给一个确定数确保随机选取的样本完全一样（无论运行多少次，无论是我们自己还是调用代码的其他人）
)

print(X_train.shape)
print(X_val.shape)
# 拆分正确，总计741份，现在训练集592份，验证集149份

(592, 5)
(149, 5)


In [49]:
!pip install torch

In [74]:
# PCA处理之后的文件格式为numpy数组
# 但 PyTorch 神经网络只吃 Tensor（张量） 这种专用格式。
# 数组主要用于通用的数学计算，运行在 CPU 上。
# 张量就是增强版本的数组。能在GPU上加速运行并且能自动计算导数(用于训练时的反向传播)。
# 所以我们要转换numpy格式到Tensor格式
import torch    #pytorch的核心库，提供最基础的数据结构——Tensor(张量)
import torch.nn as nn #torch中的神经网络子模块
import torch.optim as optim

In [60]:
# 9. 将数据转换为 PyTorch 的 Tensor 格式

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# torch.tensor()：这是 PyTorch 中最常用的创建张量的函数。它会复制一份 NumPy 数据，并将其封装成一个 Tensor 对象。、
# dtype=torch.float32：这是指定数据类型为 32位浮点数。
# 神经网络内部的权重（Weights）默认通常是 float32。如果输入是 float64（NumPy 的默认值），计算时会因为类型不匹配而报错。

y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1) # 变成 (N, 1) 的形状
# .unsqueeze(1)调整标签 (y) 的维度。由一维向量拉成了二维矩阵。变成了741行，1列的二维矩阵。
# 二维不在乎有几行几列，关键是得有行、列这两个维度
#之所以即便只有一列结果（标签）也要拉成二维，不是因为数据多，而是因为神经网络的计算本质是矩阵乘法。
#如果你把结果写成一维的 [741]，神经网络会很困惑：它分不清这 741 个数字到底是“1 个人的 741 个特征”，还是“741 个人的 1 个特征”。
# 拉成 [741, 1] 后：它明确知道了——这是 741 个人，每个人对应 1 个结果。


X_val_tensor = torch.tensor(X_val,dtype=torch.float32)
y_val_tensor = torch.tensor(y_val,dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_pca, dtype=torch.float32)

print(f"输入张量形状: {X_train_tensor.shape}") 
print(f"标签张量形状: {y_train_tensor.shape}")

输入张量形状: torch.Size([741, 5])
标签张量形状: torch.Size([741, 1])


In [83]:
# 10. 加载并“冻结”预训练神经网络模型,整合定义自己的Full完整模型

#神经网络：前向传播算误差，反向传播改参数

# 10.1. 定义官方提供的预训练模型结构 （得和预训练神经网络里的结构一样）
class PretrainedNetwork(nn.Module): ### PretrainedNetwork继承了nn.Module，能使用其一切的功能。没有继承则就是一个名为PretrainedNetwork的空壳
    def __init__(self): ##定义预训练神经网络
        super(PretrainedNetwork, self).__init__()  ##继承仪式，从PretrainedNetwork开始找父类，创建self实例，_init_()初始化函数
        
        self.fc1 = nn.Linear(5, 10)  
        #fc：fully connected 表示全连接 ：上层的神经元和下层的全部连接
        #Linear代表线性层：强调它的数学本质是 y = W·x + b的线性变换。
        #fc1:代表第一层全连接层。在这里，它负责把 5 个特征映射到 10 个隐藏特征。
        
        self.fc2 = nn.Linear(10, 3) # 隐藏 10 -> 输出 3
        #fc2:代表第二层全连接层。在这里，它负责把 10个隐藏特征浓缩到3个高阶特征。
        
        self.relu = nn.ReLU() #relu激活函数 Max(0,x)：把负数全部转化为0
    
    def forward(self, x): #定义向前传播函数
        x = self.relu(self.fc1(x)) #x是输入的特征数据
        #输入的五个特征数据先经过fc1层，再经过激活函数relu处理
        
        x = self.fc2(x) 
        #数据再经过fc2层处理，得到3个高阶特征
        return x

# 10.2 定义完整模型 (包含冻结层 + 新增层)
class MyFullModel(nn.Module):
    def __init__(self, frozen_network):
        super(MyFullModel, self).__init__()
        self.frozen_part = frozen_network
        
        # --- 在这里设计新增层 ---
        # 预训练输出是 3，所以输入必须是 3
        self.my_layers = nn.Sequential(  
            #数据通常需要按顺序经过好几个层（比如：先经过线性层，再经过激活层）。
            #如果你不想一行一行地手动写这些过程(如PretrainedNetwork中的forward函数)，
            #就可以把它们全部塞进 nn.Sequential里
            nn.Linear(3, 64),         # 增加一点宽度
            nn.BatchNorm1d(64),      
            # 加入批归一化，能让训练更稳定
            #它能让数据在每一层都保持良好的分布，防止某些神经元因为数值太大或太小而“罢工”，能让模型收敛得更深。
            nn.ReLU(),
            nn.Dropout(0.2),          # 防止过拟合：随机选择 20% 的神经元，并强行把它们的输出变成 0（即让它们暂时失效）。
            #由于每次训练时网络结构都因为随机罢工而略有不同，这相当于在训练成千上万个轻量级的小模型。
            #最后预测时，它们的效果会叠加在一起，使得结果更稳定。
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 1),          # 输出 1 个概率值
            nn.Sigmoid()              # 压缩到 0-1
        )
        
    def forward(self, x):
        # 预训练部分 (无梯度)
        with torch.no_grad():
            features = self.frozen_part(x)
        # 你的部分 (有梯度)
        output = self.my_layers(features)
        return output

In [103]:
# 11. 初始化与训练

# 实例化模型
print('We are initing the modle...')
pretrained_net = PretrainedNetwork()

# 加载你上传的权重文件 
try: ###容错机制设计，找不到文件就报错
    pretrained_net.load_state_dict(torch.load('pretrained_network.pth'))
    # pth(pytorch文件) 文件中存的是模型所有层（比如 fc1, fc2）的权重 (Weights) 和 偏置 (Biases)。这些数据被打包成了一个名为 state_dict 的 Python 字典。
    # load_state_dict 就是把这些东西加载到我们所定义的模型里面
    print("The weight loading was successful.！")
    
except:
    print("Please check the file name to ensure that the.pth file is in the current directory")

# 关键步骤：冻结参数 
# 这一步是为了满足题目要求：Do not modify or retrain the pre-trained network [cite: 52]

for param in pretrained_net.parameters():
    param.requires_grad = False
    # requires_grad：是否需要（Requires）梯度（Grad）更新
    # 模型通过计算梯度，知道某个参数（比如权重 W）是该调大一点还是调小一点，才能让误差（Loss）变小。
    # 计算梯度 = 准备更新参数。

model = MyFullModel(pretrained_net)  ###构造完整模型实体，并传入冻结层
criterion = nn.BCELoss()     ###构造损失函数
optimizer = optim.Adam(model.my_layers.parameters(), lr=0.005)  ###定义优化器

#开始全量训练
print("\nStart training with full data...")
epochs = 800 
for epoch in range(epochs):
    model.train()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# ==========================================
# 预测并生成标准 CSV (Name + Survived)
# ==========================================
print("\nThe test set is being predicted...")#正在测试测试集
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_preds = (test_outputs >= 0.5).int().numpy().flatten()

# 🔥 生成包含 Name 的标准提交文件
submission = pd.DataFrame({
    'Name': test_names,   
    'Survived': test_preds
})

# 保存
submission.to_csv('submission.csv', index=False)
print("\n✅ submission.csv has been created！")
print(submission.head())

We are initing the modle...
Please check the file name to ensure that the.pth file is in the current directory

Start training with full data...
Epoch [50/800], Loss: 0.4999
Epoch [100/800], Loss: 0.4953
Epoch [150/800], Loss: 0.4911
Epoch [200/800], Loss: 0.4857
Epoch [250/800], Loss: 0.4752
Epoch [300/800], Loss: 0.4741
Epoch [350/800], Loss: 0.4748
Epoch [400/800], Loss: 0.4713
Epoch [450/800], Loss: 0.4697
Epoch [500/800], Loss: 0.4594
Epoch [550/800], Loss: 0.4589
Epoch [600/800], Loss: 0.4514
Epoch [650/800], Loss: 0.4459
Epoch [700/800], Loss: 0.4289
Epoch [750/800], Loss: 0.4401
Epoch [800/800], Loss: 0.4281

The test set is being predicted...

✅ submission.csv has been created！
                             Name  Survived
0          Eklund, Mr. Hans Linus         0
1  McDermott, Miss. Brigdet Delia         1
2          Lefebre, Miss. Jeannie         0
3             Collyer, Mr. Harvey         0
4        Morrow, Mr. Thomas Rowan         0


In [104]:
# 保存自己训练的那部分层 (my_layers) 的状态字典
torch.save(model.my_layers.state_dict(), 'trained_added_layer_weights.pth')
print("✅ 自定义层权重已成功保存！")

✅ 自定义层权重已成功保存！
